In [1]:
import import_hack
import numpy as np
import core.steps as steps
from core import data_store as ds
from core import transformer as tr
from core.common.ds_utils import print_ds_items_info
from os.path import join as pjoin
from core.search.exhaustive_searcher import ExhaustiveSearcher
from core.quantization.pq_quantizer import PQQuantizer, restore_from_clusters, build_pq_params_str
from core.metric.asymmetric_distance_computer import AsymmetricDistanceComputer
%load_ext autoreload
%autoreload 2

In [2]:
global_descriptors_base_dir=r'C:\data\computation\brodatz\global_descriptors'

#### Setup global descriptors datastores
descriptors have been computed in global_descriptor__compute_from_image 

In [3]:
histograms_ds = ds.SQLiteTableDataStore(pjoin(global_descriptors_base_dir, 'histograms'))
lbphistograms_ds = ds.SQLiteTableDataStore(pjoin(global_descriptors_base_dir, 'lbphistograms'))
glcms_ds = ds.SQLiteTableDataStore(pjoin(global_descriptors_base_dir, 'glcms'))

# ADC exhaustive search

In [4]:
K_arr = [2 ** i for i in [4,5,6,7,8]]
m_arr = [m for m in [1, 2, 4, 8, 16]]
pq_params_arr = [{'n_clusters': K, 'n_quantizers': m} for K in K_arr for m in m_arr]

In [5]:
def adc_search(pq_params, centroids_prefix_path, pqcodes_prefix_path, query_descriptors_ds, prefix_for_neighbords_ids):
    pq_params_str = build_pq_params_str(pq_params)
    quantizer = PQQuantizer(**pq_params)

    centroids_path = centroids_prefix_path + "_" + pq_params_str + '_centroids'
    centroids_ds = ds.SQLiteTableDataStore(centroids_path)
    centroids = ds.get_as_array(centroids_ds)
    centroids = get_as_array(centroids_ds)
    
    metric = AsymmetricDistanceComputer(centroids)
    
    pqcodes_path = pqcodes_prefix_path + "_" + pq_params_str + '_pqcodes'
    pqcodes_ds = ds.SQLiteTableDataStore(pqcodes_path, ndarray_bytes_only=True)
    ids, pqcodes = ds.get_as_array(pqcodes_ds, return_ids=True)
    
    searcher_ = ExhaustiveSearcher(pqcodes, ids, metric=metric)
    n_nearest = 25

    neighbors_ids_filepath = prefix_for_neighbords_ids + "_" + pq_params_str + '_adc-neighbors-ids'
    neighbors_ids_ds = ds.CSVDataStore(neighbors_ids_filepath)
    
    steps.search_step(query_descriptors_ds, searcher_, n_nearest, neighbors_ids_ds)

In [6]:
def build_prefix_pathes(base_dir, descriptors_name):
    centroids_prefix_path=pjoin(global_descriptors_base_dir, 'centroids', descriptors_name, descriptors_name)
    pqcodes_prefix_paths=pjoin(global_descriptors_base_dir, 'pqcodes',descriptors_name,descriptors_name)
    prefix_for_neighbords_ids=pjoin(global_descriptors_base_dir, 'adc-neighbors-ids',descriptors_name,descriptors_name)
    return centroids_prefix_path, pqcodes_prefix_paths, prefix_for_neighbords_ids

In [28]:
centroids_prefix_path, pqcodes_prefix_paths, prefix_for_neighbords_ids=build_prefix_pathes(global_descriptors_base_dir, 'histograms')
for pq_params in pq_params_arr:
    adc_search(pq_params, centroids_prefix_path, pqcodes_prefix_paths, histograms_ds, prefix_for_neighbords_ids)

In [8]:
centroids_prefix_path, pqcodes_prefix_paths, prefix_for_neighbords_ids=build_prefix_pathes(global_descriptors_base_dir, 'lbphistograms')
for pq_params in pq_params_arr:
    adc_search(pq_params, centroids_prefix_path, pqcodes_prefix_paths, lbphistograms_ds, prefix_for_neighbords_ids)

In [9]:
centroids_prefix_path, pqcodes_prefix_paths, prefix_for_neighbords_ids=build_prefix_pathes(global_descriptors_base_dir, 'glcms')
for pq_params in pq_params_arr:
    adc_search(pq_params, centroids_prefix_path, pqcodes_prefix_paths, glcms_ds, prefix_for_neighbords_ids)